# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

## Follow the Directions in Bold. If you get stuck, check out the solutions lecture.

### THE DATA

** Read in the census_data.csv data with pandas**

In [1]:
import pandas as pd

In [2]:
census = pd.read_csv('../data/census_data.csv')

In [3]:
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
age               32561 non-null int64
workclass         32561 non-null object
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
gender            32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
income_bracket    32561 non-null object
dtypes: int64(5), object(9)
memory usage: 3.5+ MB


In [4]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [5]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [6]:
census['income_bracket'] = census['income_bracket'].apply(lambda label:int(label == ' >50K'))

In [7]:
"""
def label_fix(label):
    if label == ' <=50K':
        return 0
    else:
        return 1
"""

"\ndef label_fix(label):\n    if label == ' <=50K':\n        return 0\n    else:\n        return 1\n"

In [8]:
# alternative method
#census['income_bracket'] = census['income_bracket'].apply(label_fix)

### Perform a Train Test Split on the Data

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x_data = census.drop('income_bracket', axis=1)
y_data = census['income_bracket']
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=101)

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [11]:
x_data.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country'],
      dtype='object')

** Import Tensorflow **

In [12]:
import tensorflow as tf

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [13]:
workclass = tf.feature_column.categorical_column_with_hash_bucket('workclass', hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket('education', hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket('marital_status', hash_bucket_size=1000)
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation', hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket('relationship', hash_bucket_size=1000)
race = tf.feature_column.categorical_column_with_hash_bucket('race', hash_bucket_size=10000)
native_country = tf.feature_column.categorical_column_with_hash_bucket('native_country', hash_bucket_size=1000)
gender = tf.feature_column.categorical_column_with_vocabulary_list('gender',vocabulary_list=[' Male', ' Female'])

In [14]:
embedded_workclass = tf.feature_column.embedding_column(workclass, dimension=len(x_data['workclass'].unique()))
embedded_education = tf.feature_column.embedding_column(education, dimension=len(x_data['education'].unique()))
embedded_marital_status = tf.feature_column.embedding_column(marital_status, dimension=len(x_data['marital_status'].unique()))
embedded_occupation = tf.feature_column.embedding_column(occupation, dimension=len(x_data['occupation'].unique()))
embedded_relationship = tf.feature_column.embedding_column(relationship, dimension=len(x_data['relationship'].unique()))
embedded_race = tf.feature_column.embedding_column(race, dimension=len(x_data['race'].unique()))
embedded_native_country = tf.feature_column.embedding_column(native_country, dimension=len(x_data['native_country'].unique()))
embedded_gender = tf.feature_column.embedding_column(gender, dimension=len(x_data['gender'].unique()))

** Create the continuous feature_columns for the continuous values using numeric_column **

In [15]:
age = tf.feature_column.numeric_column('age')
education_num = tf.feature_column.numeric_column('education_num')
capital_gain = tf.feature_column.numeric_column('capital_gain')
capital_loss = tf.feature_column.numeric_column('capital_loss')
hours_per_week = tf.feature_column.numeric_column('hours_per_week')

** Put all these variables into a single list with the variable name feat_cols **

In [16]:
linear_feat_cols = [age, 
                    workclass, 
                    education,
                    education_num,
                    marital_status,
                    occupation, 
                    relationship, 
                    race,
                    gender,
                    capital_gain,
                    capital_loss,
                    hours_per_week, 
                    native_country]

In [17]:
dnn_feat_cols =  [age,
                  embedded_workclass, 
                  embedded_education,
                  education_num,
                  embedded_marital_status,
                  embedded_occupation, 
                  embedded_relationship, 
                  embedded_race,
                  embedded_gender,
                  capital_gain,
                  capital_loss,
                  hours_per_week, 
                  embedded_native_country]

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [18]:
linear_input_fn = tf.estimator.inputs.pandas_input_fn(x=X_train,
                                                      y=y_train,
                                                      batch_size=100,
                                                      num_epochs=None,
                                                      shuffle=True)

In [19]:
dnn_input_fn = tf.estimator.inputs.pandas_input_fn(x=X_train,
                                                   y=y_train,
                                                   batch_size=100,
                                                   num_epochs=None,
                                                   shuffle=True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [20]:
linearClassifier = tf.estimator.LinearClassifier(feature_columns=linear_feat_cols, n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/j8/4vht5fhd3rg9wv8rx18tt20c0000gp/T/tmpls4d7ab2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x138656f60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [21]:
dnnClassifier = tf.estimator.DNNClassifier(hidden_units=[13,20,20,13], feature_columns=dnn_feat_cols, n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/j8/4vht5fhd3rg9wv8rx18tt20c0000gp/T/tmp8f3_8vf0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x138656c50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


** Train your model on the data, for at least 5000 steps. **

In [22]:
linearClassifier.train(input_fn=linear_input_fn,steps=10000)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/j8/4vht5fhd3rg9wv8rx18tt20c0000gp/T/tmpls4d7ab2/model.ckpt.
INFO:tensorflow:loss = 69.31474, step = 1
INFO:tensorflow:global_step/sec: 15.4219
INFO:tensorflow:loss = 281.60962, step = 101 (6.485 sec)
INFO:tensorflow:global_step/sec: 30.7456
INFO:tensorflow:loss = 493.96362, step = 201 (

INFO:tensorflow:loss = 127.951035, step = 6001 (2.272 sec)
INFO:tensorflow:global_step/sec: 43.803
INFO:tensorflow:loss = 31.78439, step = 6101 (2.283 sec)
INFO:tensorflow:global_step/sec: 45.6176
INFO:tensorflow:loss = 181.71286, step = 6201 (2.195 sec)
INFO:tensorflow:global_step/sec: 41.9589
INFO:tensorflow:loss = 28.141228, step = 6301 (2.380 sec)
INFO:tensorflow:global_step/sec: 44.7733
INFO:tensorflow:loss = 43.35981, step = 6401 (2.232 sec)
INFO:tensorflow:global_step/sec: 41.0812
INFO:tensorflow:loss = 50.12488, step = 6501 (2.435 sec)
INFO:tensorflow:global_step/sec: 42.7206
INFO:tensorflow:loss = 44.89504, step = 6601 (2.340 sec)
INFO:tensorflow:global_step/sec: 47.1824
INFO:tensorflow:loss = 44.187756, step = 6701 (2.119 sec)
INFO:tensorflow:global_step/sec: 43.7264
INFO:tensorflow:loss = 97.55087, step = 6801 (2.287 sec)
INFO:tensorflow:global_step/sec: 45.9472
INFO:tensorflow:loss = 41.049816, step = 6901 (2.176 sec)
INFO:tensorflow:global_step/sec: 39.8358
INFO:tensorflow

In [23]:
dnnClassifier.train(input_fn=dnn_input_fn,steps=10000)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/j8/4vht5fhd3rg9wv8rx18tt20c0000gp/T/tmp8f3_8vf0/model.ckpt.
INFO:tensorflow:loss = 303.92816, step = 1
INFO:tensorflow:global_step/sec: 17.0068
INFO:tensorflow:loss = 40.18817, step = 101 (5.876 sec)
INFO:tensorflow:global_step/sec: 26.9242
INFO:tensorflow:loss = 43.90975, step = 201 (3.715 sec)
INFO:tensorflow:global_step/sec: 24.4066
INFO:tensorflow:loss = 26.708557, step = 301 (4.097 sec)
INFO:tensorfl

INFO:tensorflow:global_step/sec: 39.1726
INFO:tensorflow:loss = 24.588888, step = 6901 (2.553 sec)
INFO:tensorflow:global_step/sec: 38.4398
INFO:tensorflow:loss = 29.368475, step = 7001 (2.601 sec)
INFO:tensorflow:global_step/sec: 37.706
INFO:tensorflow:loss = 38.87306, step = 7101 (2.652 sec)
INFO:tensorflow:global_step/sec: 36.8698
INFO:tensorflow:loss = 32.3413, step = 7201 (2.712 sec)
INFO:tensorflow:global_step/sec: 36.8354
INFO:tensorflow:loss = 27.243534, step = 7301 (2.715 sec)
INFO:tensorflow:global_step/sec: 37.5172
INFO:tensorflow:loss = 31.030674, step = 7401 (2.666 sec)
INFO:tensorflow:global_step/sec: 37.8431
INFO:tensorflow:loss = 35.18019, step = 7501 (2.642 sec)
INFO:tensorflow:global_step/sec: 35.6082
INFO:tensorflow:loss = 45.310753, step = 7601 (2.809 sec)
INFO:tensorflow:global_step/sec: 37.7946
INFO:tensorflow:loss = 27.252586, step = 7701 (2.646 sec)
INFO:tensorflow:global_step/sec: 39.7049
INFO:tensorflow:loss = 27.85239, step = 7801 (2.518 sec)
INFO:tensorflow:

### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [24]:
eval_input_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,
                                                    y=y_test,
                                                    batch_size=10,
                                                    num_epochs=1, 
                                                    shuffle=False)

In [25]:
linear_result = linearClassifier.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-12T01:48:45Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /var/folders/j8/4vht5fhd3rg9wv8rx18tt20c0000gp/T/tmpls4d7ab2/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-12-01:48:58
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.85402805, accuracy_baseline = 0.7611833, auc = 0.9052077, auc_precision_recall = 0.96720517, average_loss = 0.33196247, global_step = 10000, label/mean = 0.7611833, loss = 3.319285, precision = 0.86592793, prediction/mean = 0.8018688, recall = 0.9562937
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /var/folders/

In [26]:
dnn_result = dnnClassifier.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-12T01:49:01Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/j8/4vht5fhd3rg9wv8rx18tt20c0000gp/T/tmp8f3_8vf0/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-12-01:49:13
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.8575084, accuracy_baseline = 0.7611833, auc = 0.9103053, auc_precision_recall = 0.9698105, average_loss = 0.31547186, global_step = 10000, label/mean = 0.7611833, loss = 3.1543956, precision = 0.881662, prediction/mean = 0.7634324, recall = 0.9388112
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /var/folders/j8/4vht5fhd3rg9wv8rx18tt20c0000gp/T/tmp8f3_8vf0/model.ckpt-10000


In [27]:
linear_result

{'accuracy': 0.85402805,
 'accuracy_baseline': 0.7611833,
 'auc': 0.9052077,
 'auc_precision_recall': 0.96720517,
 'average_loss': 0.33196247,
 'label/mean': 0.7611833,
 'loss': 3.319285,
 'precision': 0.86592793,
 'prediction/mean': 0.8018688,
 'recall': 0.9562937,
 'global_step': 10000}

In [28]:
dnn_result

{'accuracy': 0.8575084,
 'accuracy_baseline': 0.7611833,
 'auc': 0.9103053,
 'auc_precision_recall': 0.9698105,
 'average_loss': 0.31547186,
 'label/mean': 0.7611833,
 'loss': 3.1543956,
 'precision': 0.881662,
 'prediction/mean': 0.7634324,
 'recall': 0.9388112,
 'global_step': 10000}

In [29]:
predict_input_fn = tf.estimator.inputs.pandas_input_fn(x=X_test, 
                                                       batch_size=len(X_test), 
                                                       shuffle=False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [30]:
linear_predictions = list(linearClassifier.predict(input_fn=predict_input_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/j8/4vht5fhd3rg9wv8rx18tt20c0000gp/T/tmpls4d7ab2/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [31]:
dnn_predictions = list(dnnClassifier.predict(input_fn=predict_input_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/j8/4vht5fhd3rg9wv8rx18tt20c0000gp/T/tmp8f3_8vf0/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Each item in your list will look like this: **

In [32]:
linear_predictions[0]

{'logits': array([1.3011456], dtype=float32),
 'logistic': array([0.78602767], dtype=float32),
 'probabilities': array([0.2139723, 0.7860277], dtype=float32),
 'class_ids': array([1]),
 'classes': array([b'1'], dtype=object)}

In [33]:
dnn_predictions[0]

{'logits': array([0.8584721], dtype=float32),
 'logistic': array([0.7023413], dtype=float32),
 'probabilities': array([0.29765868, 0.7023414 ], dtype=float32),
 'class_ids': array([1]),
 'classes': array([b'1'], dtype=object)}

** Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions you will use to compare against the real y_test values. **

In [34]:
linear_predict_result = []
for predict in linear_predictions:
    linear_predict_result.append(predict['class_ids'][0])

In [35]:
dnn_predict_result = []
for predict in dnn_predictions:
    dnn_predict_result.append(predict['class_ids'][0])

In [36]:
linear_predict_result[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [37]:
dnn_predict_result[:10]

[1, 1, 1, 1, 0, 1, 1, 1, 1, 1]

** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [38]:
from sklearn.metrics import classification_report

In [39]:
print(classification_report(y_test, linear_predict_result))

              precision    recall  f1-score   support

           0       0.79      0.53      0.63      2333
           1       0.87      0.96      0.91      7436

    accuracy                           0.85      9769
   macro avg       0.83      0.74      0.77      9769
weighted avg       0.85      0.85      0.84      9769



In [40]:
print(classification_report(y_test, dnn_predict_result))

              precision    recall  f1-score   support

           0       0.75      0.60      0.67      2333
           1       0.88      0.94      0.91      7436

    accuracy                           0.86      9769
   macro avg       0.82      0.77      0.79      9769
weighted avg       0.85      0.86      0.85      9769



# Great Job!